In [46]:
import requests
#get data index from piserver
URL= "http://145.94.228.143:5000/getIndex"
r = requests.get(url=URL)
data= r.json()
print(data)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 145.94.228.143:5000


ConnectionError: HTTPConnectionPool(host='145.94.228.143', port=5000): Max retries exceeded with url: /getIndex (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11de8ab10>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [47]:
from dcd.entities.thing import Thing
from dotenv import load_dotenv
import os

# Connect to DCD Hub
load_dotenv()

THING_ID = os.environ['THING_ID']
THING_TOKEN = os.environ['THING_TOKEN']
my_thing = Thing(thing_id=THING_ID, token=THING_TOKEN)

my_thing.read()
my_thing.to_json()

INFO:dcd:things:wheelchair_speed-e706:Initialising MQTT connection for Thing 'dcd:things:wheelchair_speed-e706'
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dwd.tudelft.nl:443
INFO:dcd:things:wheelchair_speed-e706:Connection successful
DEBUG:urllib3.connectionpool:https://dwd.tudelft.nl:443 "GET /api/things/dcd:things:wheelchair_speed-e706 HTTP/1.1" 200 6724


DigiCertCA.crt exist.
{'thing': {'id': 'dcd:things:wheelchair_speed-e706', 'name': 'Wheelchair_speed', 'description': '', 'type': 'Speed', 'properties': [{'type': 'THREE_DIMENSIONS', 'name': 'My Random Property', 'description': '', 'dimensions': [{'name': 'Value1', 'description': '', 'unit': ''}, {'name': 'Value2', 'description': '', 'unit': ''}, {'name': 'Value3', 'description': '', 'unit': ''}, {'name': 'Value1', 'description': '', 'unit': ''}, {'name': 'Value2', 'description': '', 'unit': ''}, {'name': 'Value3', 'description': '', 'unit': ''}], 'id': 'my-random-property-5f3c', 'classes': [], 'values': [], 'entityId': 'dcd:things:wheelchair_speed-e706', 'readAt': 1572858284487, 'registeredAt': 1571392699000}, {'type': 'ONE_DIMENSION', 'name': 'one', 'description': '', 'dimensions': [{'name': 'Value', 'description': '', 'unit': ''}], 'id': 'one-28d7', 'classes': [], 'values': [], 'entityId': 'dcd:things:wheelchair_speed-e706', 'readAt': 1572858284487, 'registeredAt': 1571994791000}, {

{'id': 'dcd:things:wheelchair_speed-e706',
 'name': 'Wheelchair_speed',
 'description': '',
 'type': 'Speed',
 'properties': [{'id': 'my-random-property-5f3c',
   'name': 'My Random Property',
   'description': '',
   'type': 'THREE_DIMENSIONS',
   'dimensions': [{'name': 'Value1', 'description': '', 'unit': ''},
    {'name': 'Value2', 'description': '', 'unit': ''},
    {'name': 'Value3', 'description': '', 'unit': ''},
    {'name': 'Value1', 'description': '', 'unit': ''},
    {'name': 'Value2', 'description': '', 'unit': ''},
    {'name': 'Value3', 'description': '', 'unit': ''}]},
  {'id': 'one-28d7',
   'name': 'one',
   'description': '',
   'type': 'ONE_DIMENSION',
   'dimensions': [{'name': 'Value', 'description': '', 'unit': ''}]},
  {'id': 'random-shit-aa03',
   'name': 'RANDOM SHIT',
   'description': '',
   'type': 'THREE_DIMENSIONS',
   'dimensions': [{'name': 'Value1', 'description': '', 'unit': ''},
    {'name': 'Value2', 'description': '', 'unit': ''},
    {'name': 'Val

In [48]:
from datetime import datetime, timedelta
# What dates?
start_time = data[-1][-2]
end_time = data[-1][-1]

#START_DATE = datetime.strftime(now_minus_1, "%Y-%m-%d %H:%M:%S")
#END_DATE = datetime.strftime(now, "%Y-%m-%d %H:%M:%S")
print(start_time)
print(end_time)

DATE_FORMAT = '%Y-%m-%d %H:%M:%S'
from_ts = datetime.timestamp(datetime.strptime(start_time, DATE_FORMAT)) * 1000
to_ts = datetime.timestamp(datetime.strptime(end_time, DATE_FORMAT)) * 1000

PROPERTY_NAME = "Speed"

my_property = my_thing.find_property_by_name(PROPERTY_NAME)
my_property.read(from_ts, to_ts)

1572857507875
0


TypeError: strptime() argument 1 must be str, not int

In [ ]:
print(len(my_property.values))
my_property.values

In [ ]:
#Create CSV file from data 
import pandas as pd

ct = datetime.now().strftime('%A %-d %B %Y %H:%M')

data = my_property.values
df = pd.DataFrame(my_property.values, columns = ['time', 'velocity'])
df.to_csv('Workout '+ct+'.csv')
print(ct)

In [ ]:
#update Rootfolder.csv
df_root = pd.read_csv('Rootfolder.csv')
newdf_root = pd.DataFrame({'filename' : ['Workout '+ct+'.csv']})
UPDATE=df_root.append(newdf_root, ignore_index=True)
UPDATE.to_csv('Rootfolder.csv', index=False)


In [49]:
#Display different plots
import ipywidgets as widgets
from ipywidgets import interactive, interact, interact_manual
import plotly.express as px

df_root = pd.read_csv('Rootfolder.csv')

def selected_workout(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, "workout")
    return unique

dropdown_file = widgets.Dropdown(options = selected_workout(df_root.filename))

output_file = widgets.Output()

def dropdown_file_eventhandler(change):
    output_file.clear_output()
    with output_file:
        if (change.new=="Workout"):
            print('select your workout')
        else:
            global df_plot
            df_plot=pd.read_csv(change.new)
            fig1 = px.line(df_plot, x = 'time', y = 'velocity', title='Workout ')
            fig1.show()

dropdown_file.observe(dropdown_file_eventhandler, names='value')

display(dropdown_file)


Dropdown(options=('workout', 'Workout Monday 28 October 2019 1530.csv', 'Workout Monday 4 November 2019 09:52.…

In [50]:
display(output_file)

Output()

In [ ]:
#show statistics
import numpy as np
from scipy.stats import kurtosis, skew

average_speed = np.mean(df_plot['velocity'])
average_speed